## 1. Import Library and define function 

In [128]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [129]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

def get_clf_eval(y_test, y_pred=None):
    confusion = pd.DataFrame(confusion_matrix(y_test, y_pred), index = ['T[0]', 'F[1]'], columns = ['pred_T[0]', 'pred_F[1]'])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [130]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx + 1
    series = series.map(my_dict)

    return series

### 1.1 load data

In [131]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/submission.csv")
df_all = pd.concat([df_train, df_test], axis = 0)
df_all.shape, df_train.shape, df_test.shape

((64570, 30), (59299, 29), (5271, 30))

In [132]:
df_all.isnull().sum()

bant_submit                    0
customer_country             982
business_unit                  0
com_reg_ver_win_rate       48214
customer_idx                   0
customer_type              45418
enterprise                     0
historical_existing_cnt    49539
id_strategic_ver           60533
it_strategic_ver           63396
idit_strategic_ver         59359
customer_job               20172
lead_desc_length               0
inquiry_type                2233
product_category           21232
product_subcategory        54542
product_modelname          54779
customer_country.1           982
customer_position              0
response_corporate             0
expected_timeline          33271
ver_cus                        0
ver_pro                        0
ver_win_rate_x             43780
ver_win_ratio_per_bu       47360
business_area              43780
business_subarea           57228
lead_owner                     0
is_converted                5271
id                         59299
dtype: int

## 2. Data preprocessing

### 2.1 drop columns

In [133]:
# 열 삭제
drop_col = ['customer_country.1', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver',
            'product_subcategory', 'product_modelname', 'business_area', 'business_subarea', 'ver_cus', 'ver_pro']

df_all.drop(columns = drop_col, inplace = True)

### 2.2 결측치 처리(수치형 데이터)

In [134]:
# 결측값 0으로 넣을 컬럼
fillna_col = ['com_reg_ver_win_rate', 'historical_existing_cnt',
             'ver_win_rate_x', 'ver_win_ratio_per_bu']

for col in fillna_col:
    df_all[col] = df_all[col].fillna(0)

### 2.3 문자형 변수 처리

In [135]:
df_all['customer_type'] = df_all['customer_type'].str.lower()
df_all['customer_type'] = df_all['customer_type'].replace(['etc.', 'other', 'others'], 'etc')
df_all['customer_type'] = df_all['customer_type'].replace(['end-customer', 'end customer', 'end-user', 'commercial end-user'], 'end_user')
df_all['customer_type'] = df_all['customer_type'].replace(['specifier/ influencer', 'specifier / influencer'], 'influencer')
df_all['customer_type'] = df_all['customer_type'].replace(['homeowner', 'home owner'], 'home_owner')
df_all['customer_type'] = df_all['customer_type'].replace(['software/solution provider', 'software / solution provider'], 'solution_provider')
df_all['customer_type'] = df_all['customer_type'].replace('hvac engineer', 'engineer')
df_all['customer_type'] = df_all['customer_type'].replace('dealer/distributor', 'distributor')
df_all['customer_type'] = df_all['customer_type'].replace('architect/consultant', 'consultant')
df_all['customer_type'] = df_all['customer_type'].replace('installer/contractor', 'installer')
df_all['customer_type'] = df_all['customer_type'].replace('technical assistant', 'technician')
df_all['customer_type'].fillna('etc', inplace = True)

In [136]:
df_all['customer_type'].value_counts()

etc                     45438
end_user                13106
influencer               3356
channel partner          1695
service partner           447
solution eco-partner      292
installer                  57
engineer                   43
corporate                  31
consultant                 20
developer                  18
technician                 17
home_owner                 15
manager / director          8
solution_provider           8
distributor                 6
reseller                    5
interior designer           5
system integrator           2
administrator               1
Name: customer_type, dtype: int64

In [137]:
product = ['video wall', 'led signage', 'standalone', 'window facing product', 'pantallas interactivas para clinicas',
           'tv interactive', 'display product', 'high inch 86 / 98 or 110', 'display textbook and photos', 'digital platform',
           'hospital tv', 'one quick:flex', 'hotel tv products', 'educational equipments']

etc = ['etc.', 'other', 'other_', 'others']

sales_inquiry = ['sales inquiry', 'sales', 'probeam precio', 'first info and pricing']

quotation_purchase_consultation = ['quotation or purchase consultation', 'request for quotation or purchase', 
                                      'quotation_or_purchase_consultation', 'purchase or quotation', 
                                      'quotation_', 'purchase', 'tôi cần tham khảo giá và giải pháp từ lg',
                                      'vui lòng báo giá giúp mình sản phẩm đo thân nhiệt xin cảm ơn',
                                      'solicito apoyo para realizar cotizacion de los dispositivos que ofrecen en la solución\xa0one quick:\xa0']

usage_technical_consultation = ['usage or technical consultation', 'technical consultation', 
                                'request for technical consulting', 'usage_or_technical_consultation', 
                                'technical_consultation', 'technical',
                                'preciso de um monitor médico para radiografia convencional e tomogrtafia.']

product_information = ['i want to know the details about it','toi muon tim hieu thong tin ky thuat, gia ca cua sp de su dung']

In [138]:
df_all['inquiry_type'] = df_all['inquiry_type'].str.lower()
df_all['inquiry_type'].replace(product, 'product', inplace = True)
df_all['inquiry_type'].replace(etc, 'etc', inplace = True)
df_all['inquiry_type'].replace(sales_inquiry, 'sales inquiry', inplace = True)
df_all['inquiry_type'].replace(quotation_purchase_consultation, 'quotation_or_purchase_consultation', inplace = True)
df_all['inquiry_type'].replace(usage_technical_consultation, 'usage or technical consultation', inplace = True)
df_all['inquiry_type'].replace(product_information, 'product information', inplace = True)
df_all['inquiry_type'].fillna('etc', inplace = True)

In [139]:
df_all['inquiry_type'].value_counts()

quotation_or_purchase_consultation                                                                                     45854
sales inquiry                                                                                                          10084
etc                                                                                                                     3756
usage or technical consultation                                                                                         1856
product information                                                                                                     1308
trainings                                                                                                                455
services                                                                                                                 449
request for partnership                                                                                                  360


In [140]:
country_processing = pd.read_csv('country_processing.csv')
df_all['country'] = country_processing ['country1']
df_all.drop(columns = ['customer_country'], inplace = True)

In [141]:
df_all['country'].value_counts()

india          19717
brazil          8824
usa             5786
mexico          2818
philippines     2719
               ...  
st maarten         1
monaco             1
haiti              1
isle of man        1
uzbekistan         1
Name: country, Length: 166, dtype: int64

In [142]:
expected_timeline = pd.read_csv('expected_timeline.csv')
df_all['expected_timeline'] = expected_timeline['expected_timeline']

In [143]:
cat_position = pd.read_csv('product_category, customer_position.csv')
df_all['customer_position'] = expected_timeline['customer_position']
df_all.drop(columns ='product_category', inplace = True)

In [144]:
df_all['customer_position'].value_counts()

none                                                    20709
manager                                                  9245
ceo/founder                                              9133
other                                                    6812
director                                                 5543
                                                        ...  
pgt physics                                                 1
education professional                                      1
chemistry teacher                                           1
director cum faculty at gaining apex coaching centre        1
radiology professional                                      1
Name: customer_position, Length: 117, dtype: int64

In [145]:
df_all.isnull().sum()

bant_submit                    0
business_unit                  0
com_reg_ver_win_rate           0
customer_idx                   0
customer_type                  0
enterprise                     0
historical_existing_cnt        0
customer_job               20172
lead_desc_length               0
inquiry_type                   0
customer_position              0
response_corporate             0
expected_timeline              0
ver_win_rate_x                 0
ver_win_ratio_per_bu           0
lead_owner                     0
is_converted                5271
id                         59299
country                        0
dtype: int64

In [146]:
# 전처리 필요한 4개 컬럼 간단하게
value_counts = df_all['customer_job'].value_counts()
values_to_replace = value_counts[value_counts == 1].index
df_all['customer_job'] = df_all['customer_job'].apply(lambda x: 'other' if x in values_to_replace else x)
df_all['customer_job'].fillna('other', inplace = True)

value_counts = df_all['customer_type'].value_counts()
values_to_replace = value_counts[value_counts == 1].index
df_all['customer_type'] = df_all['customer_type'].apply(lambda x: 'etc' if x in values_to_replace else x)

value_counts = df_all['customer_position'].value_counts()
values_to_replace = value_counts[value_counts == 1].index
df_all['customer_position'] = df_all['customer_position'].apply(lambda x: 'etc' if x in values_to_replace else x)

value_counts = df_all['country'].value_counts()
values_to_replace = value_counts[value_counts == 1].index
df_all['country'] = df_all['country'].apply(lambda x: 'etc' if x in values_to_replace else x)

value_counts = df_all['inquiry_type'].value_counts()
values_to_replace = value_counts[value_counts == 1].index
df_all['inquiry_type'] = df_all['inquiry_type'].apply(lambda x: 'etc' if x in values_to_replace else x)

In [147]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64570 entries, 0 to 5270
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              64570 non-null  float64
 1   business_unit            64570 non-null  object 
 2   com_reg_ver_win_rate     64570 non-null  float64
 3   customer_idx             64570 non-null  int64  
 4   customer_type            64570 non-null  object 
 5   enterprise               64570 non-null  object 
 6   historical_existing_cnt  64570 non-null  float64
 7   customer_job             64570 non-null  object 
 8   lead_desc_length         64570 non-null  int64  
 9   inquiry_type             64570 non-null  object 
 10  customer_position        64570 non-null  object 
 11  response_corporate       64570 non-null  object 
 12  expected_timeline        64570 non-null  object 
 13  ver_win_rate_x           64570 non-null  float64
 14  ver_win_ratio_per_bu   

In [148]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "country",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

In [149]:
df_train = df_all.iloc[:len(df_train)]
df_test = df_all.iloc[len(df_train):]

In [125]:
df_train.shape, df_test.shape

((59299, 19), (5271, 19))

In [159]:
df_train.is_converted.sum()

4850.0

In [176]:
df_train[df_train['is_converted'] == True].customer_idx.value_counts().head(20)

25096    2421
47466      67
14746      38
42108      19
7740       17
45319      15
9324       14
26337      13
5023       11
9857       11
27735      10
8582       10
19251       8
19252       8
10350       7
11515       7
5763        6
14141       6
13440       6
13580       6
Name: customer_idx, dtype: int64

In [183]:
df_train[df_train['customer_idx'] == 25096].index

Int64Index([  405,   408,   429,   430,   433,  1741,  1742,  1744,  1745,
             1746,
            ...
            48008, 48009, 48010, 48011, 48012, 48013, 48015, 48023, 48024,
            53491],
           dtype='int64', length=2421)

In [190]:
df_test[df_test['customer_idx'] == 9324                                          ]

,bant_submit,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,customer_job,lead_desc_length,inquiry_type,customer_position,response_corporate,expected_timeline,ver_win_rate_x,ver_win_ratio_per_bu,lead_owner,is_converted,id,country
32,0.25,4,0.0,9324,16,1,0.0,124,117,9,41,51,1,0.000013,0.0,375,NaN,9967.0,123
125,0.25,4,0.0,9324,16,1,0.0,124,117,9,13,51,6,0.000013,0.0,375,NaN,10020.0,44
137,0.25,4,0.0,9324,16,1,0.0,124,117,9,41,51,4,0.000013,0.0,375,NaN,9966.0,65
142,0.25,4,0.0,9324,16,1,0.0,124,117,9,41,51,1,0.000013,0.0,375,NaN,9959.0,47
207,0.25,4,0.0,9324,16,1,0.0,124,117,9,41,51,3,0.000013,0.0,375,NaN,9949.0,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5132,0.25,4,0.0,9324,16,1,0.0,124,117,9,47,51,4,0.000013,0.0,375,NaN,9933.0,149
5183,0.25,4,0.0,9324,16,2,0.0,124,117,9,47,51,4,0.000013,0.0,375,NaN,10023.0,47
5214,0.25,4,0.0,9324,16,1,0.0,124,117,9,47,51,4,0.000013,0.0,375,NaN,10028.0,152
5248,0.25,4,0.0,9324,16,1,0.0,124,117,9,47,51,4,0.000013,0.0,375,NaN,9946.0,152


In [158]:
df_train[df_train.customer_idx == 25096]

,bant_submit,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,customer_job,lead_desc_length,inquiry_type,customer_position,response_corporate,expected_timeline,ver_win_rate_x,ver_win_ratio_per_bu,lead_owner,is_converted,id,country
405,1.00,3,0.075000,25096,8,2,0.0,59,112,9,13,42,4,0.003079,0.064566,153,1.0,NaN,123
408,1.00,3,0.075000,25096,8,1,0.0,163,147,9,13,42,4,0.003079,0.064566,153,1.0,NaN,123
429,0.75,3,0.075000,25096,8,2,0.0,85,106,17,41,42,4,0.003079,0.064566,153,1.0,NaN,123
430,0.50,3,0.075000,25096,8,2,0.0,177,264,9,50,42,6,0.003079,0.064566,153,1.0,NaN,123
433,0.50,3,0.075000,25096,8,2,0.0,85,85,9,47,42,4,0.003079,0.064566,153,1.0,NaN,123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48013,0.25,4,0.000000,25096,8,1,0.0,36,60,9,47,37,6,0.000000,0.000000,240,1.0,NaN,58
48015,0.25,4,0.000000,25096,8,1,0.0,178,74,9,47,37,6,0.000000,0.000000,240,1.0,NaN,58
48023,0.25,4,0.000000,25096,8,1,0.0,161,35,9,47,42,6,0.000000,0.000000,785,1.0,NaN,123
48024,0.25,4,0.000000,25096,8,1,0.0,124,19,9,47,18,6,0.000000,0.000000,819,1.0,NaN,51


In [216]:
# df_all.to_csv('pre_processing.csv', index = 0)

## 3. modeling

### 3.1 UnderSampling

In [126]:
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.under_sampling import RandomUnderSampler

X_resampled, y_resampled = RandomUnderSampler(random_state=42).fit_resample(df_train.drop(["is_converted", 'id'], axis=1), df_train["is_converted"].astype(int))

In [127]:
x_train, x_val, y_train, y_val = train_test_split(
    X_resampled,
    y_resampled,
    stratify=y_resampled,
    test_size=0.2,
    random_state=42,
)